In [1]:
import urlparse
from tablescraper import paginate_url, scrape_elems, dfs_from_tables, combine_df_rows, html_to_string, url_params
from IPython.core.display import display, HTML

In [2]:
actor_page_urls = paginate_url(
    'http://www.boxofficemojo.com/people/?view=Actor&pagenum={pagenum}&sort=sumgross&order=DESC&&p=.htm', 16)

actor_page_tables = scrape_elems(actor_page_urls, '//table//table', elem_index=0)

actor_page_dfs = dfs_from_tables(actor_page_tables,
    header=['rank','actor','total_gross','no_movies','avg_gross','top_picture','top_gross'])

actor_gross = combine_df_rows(actor_page_dfs, ignore_index=True)

pre_dedup_len = len(actor_gross)

# remove duplicates
actor_gross.drop_duplicates(subset='rank', keep='first', inplace=True)

print '\n Done. Got %i rows from %i pages (%i duplicates removed)' % (
    len(actor_gross), len(actor_page_urls), pre_dedup_len)

actor_gross.head()


 Done. Got 781 rows from 16 pages (3448 duplicates removed)


,rank,actor,total_gross,no_movies,avg_gross,top_picture,top_gross
0,1,Harrison Ford,"$4,870.6",41,$118.8,Star Wars: The Force Awakens,$935.6
1,2,Samuel L. Jackson,"$4,646.7",68,$68.3,The Avengers,$623.4
2,3,Morgan Freeman,"$4,377.3",59,$74.2,The Dark Knight,$534.9
3,4,Tom Hanks,"$4,336.5",43,$100.8,Toy Story 3,$415.0
4,5,Eddie Murphy,"$3,810.4",38,$100.3,Shrek 2,$441.2


In [3]:
# selects the table 'Adjusted for Ticket Price Inflation'
# adjusted_gross_xpath = '(//table)[5]'
adjusted_gross_xpath = '//*[text()="Adjusted for Ticket Price Inflation"]/parent::node()/following::table[1]'

harrison_ford_table = scrape_elems(
    'http://www.boxofficemojo.com/people/chart/?view=Actor&id=harrisonford.htm', 
    adjusted_gross_xpath, elem_index=0)

actor_info_headers = 'Rank Title Studio Adjusted_Gross Unadjusted_Gross Release'.lower().split()

harrison_ford_df = dfs_from_tables(harrison_ford_table,
    header=actor_info_headers)[0]

print 'Got %i rows' % len(harrison_ford_df)
harrison_ford_df.head()

Got 48 rows


,rank,title,studio,adjusted_gross,unadjusted_gross,release
0,1,Star Wars,Fox,"$1,549,640,500","$460,998,007",5/25/77
1,2,Star Wars: The Force Awakens,BV,"$935,585,300","$935,585,257",12/18/15
2,3,The Empire Strikes Back,Fox,"$854,171,500","$290,475,067",5/21/80
3,4,Return of the Jedi,Fox,"$818,316,900","$309,306,177",5/25/83
4,5,Raiders of the Lost Ark,Par.,"$770,183,000","$248,159,971",6/12/81


# Get links to all the actors' pages

In [4]:
actor_links = [actor_link 
   for page_table in actor_page_tables 
   for actor_link in page_table.xpath('((//table)[3]//td[2])[position()>1]//a/@href')]

print actor_links[-10:]
print len(actor_links)
[i for i, x in enumerate(actor_links) if 'josephschneider' in x]

['./chart/?view=Actor&id=morganspurlock.htm', './chart/?view=Actor&id=maconblair.htm', './chart/?view=Actor&id=mikenichols.htm', './chart/?view=Actor&id=davidwain.htm', './chart/?view=Actor&id=sebastiansilva.htm', './chart/?view=Actor&id=leighwhannell.htm', './chart/?view=Actor&id=eliroth.htm', './chart/?view=Actor&id=jaychandrasekhar.htm', './chart/?view=Actor&id=alexrossperry.htm', './chart/?view=Actor&id=josephschneider.htm']
3448


[2180, 2511, 2792, 3023, 3204, 3335, 3416, 3447]

In [5]:
actor_rows_per_page = [len(page_table.xpath('((//table)[3]//td[2])[position()>1]//a/@href'))
                             for page_table in actor_page_tables]
print actor_rows_per_page
print sum(actor_rows_per_page)

[50, 100, 150, 200, 250, 300, 350, 400, 381, 331, 281, 231, 181, 131, 81, 31]
3448


Wow, it turns out that Box Office Mojo's pagination is broken. Each page is supposed to have 50 rows, but instead has some random number of rows usually over 100. The next page overlaps the previous. This is a mess!

Since I only care about getting the links, I can just make a set

In [6]:
actor_links_set = set(actor_links)
print len(actor_links_set)
# As of 2016-04-13, this number should be 781

781


In [10]:
base_url = 'http://www.boxofficemojo.com/people/'
actor_links_abs = [urlparse.urljoin(base_url, rel_link) for rel_link in actor_links_set]
print len(actor_links_abs)
print actor_links_abs[:10]

781
['http://www.boxofficemojo.com/people/chart/?view=Actor&id=paulbettany.htm', 'http://www.boxofficemojo.com/people/chart/?view=Actor&id=katebosworth.htm', 'http://www.boxofficemojo.com/people/chart/?view=Actor&id=christinetaylor.htm', 'http://www.boxofficemojo.com/people/chart/?view=Actor&id=kathybates.htm', 'http://www.boxofficemojo.com/people/chart/?view=Actor&id=jackdavenport.htm', 'http://www.boxofficemojo.com/people/chart/?view=Actor&id=violadavis.htm', 'http://www.boxofficemojo.com/people/chart/?view=Actor&id=umathurman.htm', 'http://www.boxofficemojo.com/people/chart/?view=Actor&id=evamendes.htm', 'http://www.boxofficemojo.com/people/chart/?view=Actor&id=melissabenoist.htm', 'http://www.boxofficemojo.com/people/chart/?view=Actor&id=jessicabiel.htm']


Finally, actor_links_abs has all the actor page links. The next step is to scrape the 'Adjusted for Ticket Price Inflation' tables from each actor page using those links.

In [8]:
actor_revenues_tables = scrape_elems(actor_links_abs, adjusted_gross_xpath, 0, actor_info_headers)

actor_revenues_dfs = dfs_from_tables(actor_revenues_tables,
    header=actor_info_headers)

# Create actor_id column using actor's unique URL id
for actor_link, df in zip(actor_links_abs, actor_revenues_dfs):
    df['actor_id'] = url_params(actor_link)['id'][0]

actor_revenue = combine_df_rows(actor_revenues_dfs, ignore_index=True)

print 'Got %i rows' % len(actor_revenue)
print '%i unique actors scraped' % len(actor_revenue.actor_id.unique())
actor_revenue.head()

Got 16416 rows


,rank,title,studio,adjusted_gross,unadjusted_gross,release,actor_id
0,-,Marvel's The Avengers,BV,"$668,866,600","$623,357,910",5/4/12,paulbettany.htm
1,1,Avengers: Age of Ultron,BV,"$465,684,200","$459,005,868",5/1/15,paulbettany.htm
2,-,Iron Man 3,BV,"$424,632,700","$409,013,994",5/3/13,paulbettany.htm
3,-,Iron Man,Par.,"$385,808,100","$318,412,101",5/2/08,paulbettany.htm
4,-,Iron Man 2,Par.,"$341,908,200","$312,433,331",5/7/10,paulbettany.htm


In [13]:
actor_revenue[actor_revenue['title'] == 'Iron Man']

,rank,title,studio,adjusted_gross,unadjusted_gross,release,actor_id
3,-,Iron Man,Par.,"$385,808,100","$318,412,101",5/2/08,paulbettany.htm
1549,-,Iron Man,Par.,"$385,808,100","$318,412,101",5/2/08,jonfavreau.htm
3343,1,Iron Man,Par.,"$385,808,100","$318,412,101",5/2/08,terrencehoward.htm
4114,2,Iron Man,Par.,"$385,808,100","$318,412,101",5/2/08,clarkgregg.htm
4668,-,Iron Man,Par.,"$385,808,100","$318,412,101",5/2/08,samuelljackson.htm
7883,1,Iron Man,Par.,"$385,808,100","$318,412,101",5/2/08,jeffbridges.htm
8099,4,Iron Man,Par.,"$385,808,100","$318,412,101",5/2/08,robertdowneyjr.htm
8651,2,Iron Man,Par.,"$385,808,100","$318,412,101",5/2/08,gwynethpaltrow.htm
